In [1]:
import pandas as pd
import numpy as np
import json, urllib, ast
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout

In [2]:
data = pd.read_csv('data/links_erl.csv')
with open('data/all_edges_2010.json') as afile:
    all_edges = json.load(afile)
    all_edges = {ast.literal_eval(k): v for k, v in all_edges.items()}
with open('data/completed_edges_2010.json') as cfile:
    complete_edges = json.load(cfile)
with open('data/data_dict_utility.json') as dfile:
    data_dict = json.load(dfile)

In [3]:
data_dict

{'basin_name': {'ALEXANDER VALLEY': {'subbasin_name': 'CLOVERDALE AREA',
   'basin_code': 'GW1801-1-54',
   'subbasin_code': 'GW1801-1-54.02'},
  'EEL RIVER VALLEY': {'subbasin_name': '',
   'basin_code': 'GW1801-1-10',
   'subbasin_code': 'GW1801-1-10'},
  'EUREKA PLAIN': {'subbasin_name': '',
   'basin_code': 'GW1801-1-09',
   'subbasin_code': 'GW1801-1-09'},
  'LOWER RUSSIAN RIVER VALLEY': {'subbasin_name': '',
   'basin_code': 'GW1801-1-60',
   'subbasin_code': 'GW1801-1-60'},
  'MAD RIVER VALLEY': {'subbasin_name': 'MAD RIVER LOWLAND',
   'basin_code': 'GW1801-1-08',
   'subbasin_code': 'GW1801-1-08.01'},
  'SANTA ROSA VALLEY': {'subbasin_name': '',
   'basin_code': 'GW1806-3-36',
   'subbasin_code': 'GW1806-3-36'},
  'SMITH RIVER PLAIN': {'subbasin_name': '',
   'basin_code': 'GW1801-1-01',
   'subbasin_code': 'GW1801-1-01'},
  'UKIAH VALLEY': {'subbasin_name': '',
   'basin_code': 'GW1801-1-52',
   'subbasin_code': 'GW1801-1-52'},
  'WILSON GROVE FORMATION HIGHLANDS': {'subbasin

In [4]:
data = data[data['year'] == 2010]
data.head(5)

,year,data_year,source,target,cumulative_volume_af,transmission_kwh/af,treatment_kwh/af,used_vol_af
0,2010,2010,1801001PD,1801001E,1624.000000,163.1,0.0,1624.000000
11,2010,2010,1801007PD,1801007E,2015.200000,163.1,0.0,2015.200000
27,2010,2010,1801009PD,1801017PD,561.767505,25.0,0.0,557.003898
28,2010,2010,1801009PD,1801009E,2924.550000,163.1,0.0,2924.550000
34,2010,2010,1801011PD,1801011E,1350.000000,163.1,0.0,1350.000000


In [5]:
import build_tools as build
build.children(data, 'WWT1803033')

['WWT1803033',
 ['1803033REC',
  ['1803033NPD', ['1803033E']],
  ['1803019NPD', ['1803019E']],
  ['GWR1803-5-22.08',
   ['1803033GW', ['1803033E'], ['1803033PD', ['1803033E']]]]]]

In [9]:
from sub_network import subWESTnet

g = subWESTnet('data/links_erl.csv', 2010)

g.upstream('1803033E')

from networkx.drawing.nx_pydot import write_dot
write_dot(g.graph,'graph.dot')

Unweighted upstream graph built.


In [35]:
sank_dicts = []
for v in g.edges.values():
    sank_dict = {}
    sank_dict['source'] = v['source']
    sank_dict['target'] = v['target']
    sank_dict['value'] = v['used_vol_af']
    sank_dict['color'] = 'steelblue'
    if v['used_vol_af'] == 0:
        sank_dict['color'] = 'goldenrod'
        sank_dict['value'] = .5
    sank_dicts.append(sank_dict)

In [37]:
flows = pd.DataFrame(sank_dicts)

src = 'GW1803-5-22'
find = flows[flows['source'] == src]
total = sum(find.value.values)
targets = find['target']
target_total = 0
for i in targets:
    v_i = flows[flows.source == i].value.values
    target_total += sum(v_i)

if target_total != total:
    make_up = total - target_total
    sank_dicts.append({'source':'GW1803-5-22.08', 'target':'Other', 'value':float(make_up), 'color':'grey'})

In [38]:
SankeyWidget(layout=Layout(width="1500", height="1000"), 
             margins=dict(top=10, bottom=0, left=100, right=100),
             links=sank_dicts)

SankeyWidget(layout=Layout(height='1000', width='1500'), links=[{'source': 'SW_CVPFKC', 'target': '1803033E', …

In [138]:
gi = subWESTnet('data/links_erl.csv', 2010)
gi.downstream('WWT1803033')
gi.graph.edges.values()

sank_dicts = []
for v in gi.edges.values():
    sank_dict = {}
    sank_dict['source'] = v['source']
    sank_dict['target'] = v['target']
    sank_dict['value'] = v['used_vol_af']
    sank_dict['color'] = 'steelblue'
    if v['used_vol_af'] == 0:
        sank_dict['color'] = 'goldenrod'
        sank_dict['value'] = .00001
    sank_dicts.append(sank_dict)

Unweighted downstream graph complete.


In [139]:
flows = pd.DataFrame(sank_dicts)
in_val, out_val = {}, {}
for ind, row in flows.iterrows():
    source = row['source']
    src_targets = flows[flows.source == source]
    out_val[source] = sum(src_targets['value'])

for ind, row in flows.iterrows():
    target = row['target']
    target_srcs = flows[flows.target == target]
    in_val[target] = sum(target_srcs['value'])

In [140]:
out_val

{'WWT1803033': 88.0,
 '1803033REC': 1206.50001,
 '1803033NPD': 88.0,
 '1803019NPD': 1118.5,
 'GWR1803-5-22.08': 1e-05,
 '1803033GW': 29572.94001,
 '1803033PD': 61955.79}

In [141]:
in_val

{'1803033REC': 88.0,
 '1803033NPD': 88.0,
 '1803033E': 62043.790010000004,
 '1803019NPD': 1118.5,
 '1803019E': 1118.5,
 'GWR1803-5-22.08': 1e-05,
 '1803033GW': 1e-05,
 '1803033PD': 29572.94}

In [148]:
fill = []
for ind, row in flows.iterrows():
    src = row['source']
    trgt = row['target']
    src_total_val = out_val[src]
    trgt_total_val = in_val[src]
    
    if src_total_val < trgt_total_val:
        fill.append({'source': 'Other utilities to {}'.format(trgt),
                     'target': trgt,
                     'value': trgt_total_val - src_total_val,
                     'color': 'grey'})
    elif src_total_val > trgt_total_val:
        fill.append({'source': trgt,
                     'target': '{} to Other Utilities'.format(trgt),
                     'value': src_total_val - trgt_total_val,
                     'color': 'grey'})

KeyError: 'WWT1803033'

In [147]:
SankeyWidget(layout=Layout(width="1500", height="1000"), 
             margins=dict(top=10, bottom=0, left=100, right=100),
             links=sank_dicts)

SankeyWidget(layout=Layout(height='1000', width='1500'), links=[{'source': 'WWT1803033', 'target': '1803033REC…

In [68]:
flows = pd.DataFrame(sank_dicts)
fill = []
for ind, row in flows.iterrows():
    src = row['source']
    srcs = flows[flows.source == src]
    # for Large -> small
    for nodes in srcs.iterrows():
        node_src = node['target']
        total = sum(find.value.values)
        targets = find['target']
        target_total = 0
        for i in targets:
            v_i = flows[flows.source == i].value.values
            target_total += sum(v_i) 

        if target_total != total:
            make_up = total - target_total
            sank_dicts.append({'source': 'GW1803-5-22.08', 
                               'target': 'To other target', 
                               'value': float(make_up), 
                               'color': 'grey'})
    # for too large -> val
    #trgt = row['target'].values[0]
    #for trgt in trgts:
        

NameError: name 'node' is not defined

----

In [ ]:
1807199, 1807201, 1809023, 1807202, 1807203

In [46]:
la_ind = [i for i,v in data.iterrows() if '1809023' in v['target']]
la_ind

[5958, 6035, 8629]

In [47]:
data.loc[la_ind]

,year,data_year,source,target,cumulative_volume_af,transmission_kwh/af,treatment_kwh/af,used_vol_af
5958,2010,2010,1809003PD,1809023PD,38111.111110,25.0,0.00,38024.0
6035,2010,2010,1809023PD,1809023E,45396.000000,163.1,0.00,45396.0
8629,2010,2010,GW1809-6-44,1809023PD,7388.888889,362.0,181.75,7372.0


In [48]:
build.children(data, 'GW1809-6-44')

['GW1809-6-44',
 ['1809011PD', ['1809011E']],
 ['1809031PD', ['1809031E']],
 ['1809033PD', ['1809033E']],
 ['1809023PD', ['1809023E']],
 ['1809027PD', ['1809027E']]]

In [40]:
build.children(data, 'GW1807203a')

['GW1807203a', ['1807203PD', ['1807203E']]]

In [24]:
la_dept = WESTnet('data/links_erl.csv', 2010)
la_dept.downstream('GW1807203a')

Unweighted downstream graph complete.


------
floweaver stuff.

In [31]:
flows = pd.DataFrame(sank_dicts)
flows

,color,source,target,value
0,steelblue,SW_CVPFKC,1803033E,53121.00
1,steelblue,1803033GW,1803033PD,29572.94
2,steelblue,FRES_ID,1803033PD,18266.60
3,steelblue,SW_CVPFKC,1803033PD,14116.25
4,steelblue,RES_MLRTN,SW_CVPFKC,67371.23
5,steelblue,1803033PD,1803033E,61955.79
6,steelblue,1803033REC,1803033NPD,88.00
7,steelblue,1803033NPD,1803033E,88.00
8,steelblue,GW1803-5-22.08,1803033GW,29572.94
9,steelblue,GW1803-5-22,GW1803-5-22.08,49779.01


In [19]:
from floweaver import *

# Set the default size to fit the documentation better.
size = dict(width=570, height=300)

nodes = {
    'farms': ProcessGroup(['farm1', 'farm2', 'farm3',
                           'farm4', 'farm5', 'farm6']),
    'customers': ProcessGroup(['James', 'Mary', 'Fred', 'Susan']),
}

In [21]:
ordering = [
    ['farms'],       # put "farms" on the left...
    ['customers'],   # ... and "customers" on the right.
]

In [22]:
bundles = [
    Bundle('farms', 'customers'),
]

In [30]:
sdd = SankeyDefinition(nodes, bundles, ordering)
weave(sdd, flows).to_widget(**size)

TypeError: add_node() takes 2 positional arguments but 3 were given